In [9]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

import gensim
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV
import xgboost as xgb 
import statistics
from scipy.fftpack import fft, ifft


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler

from igraph import *

from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

import optuna

import sklearn.datasets

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Record Constants
INPUT_TRAIN_IN = "../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../data/train_out.csv"  
INPUT_TEST_IN = "../../data/test_in.csv"
INPUT_TEST_OUT = "../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


WINDOW_SIZE =  10 # Final RNA Sequence will be 101 Length 
MAX_LENGTH = (WINDOW_SIZE*2) +1


# 1 - ANF Encoding 
# 2 - One Hot Encoding  
# 3 - Complex Network  
# 4 - Word2Vec Model 
# 5 - K-mer with One Hot 
# 6 - Multi RM Encoding
# 7 - Multi Dimensional K-mer with One Hot 
ENCODING_METHOD = 3

# 1 - XGBOOST 
# 2 - XGBOOST WITH Randomize Search 
# 3 - XGBOOST WITH Grid Search 
# 4 - Random Forest 
# 5 - XGBOOST WITH Optuna Hyper Parameter Optimizatio
# 6 - LSTM 
MODEL = 6

FRAMEWORK = "PYTORCH"

INPUT_DIMENSION = 0

PERFORM_DATA_BALANCING = True
K_MERS_SIZE = 3


# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'


In [20]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1)
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1 )

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )

x_data = pd.concat([x_train_raw, x_test_raw, x_valid_raw], axis=0, ignore_index=True)
y_data = pd.concat([y_train_raw, y_test_raw, y_valid_raw], axis=0, ignore_index=True)

print(f"Train Shape of X : {x_train_raw.shape} and Tranin Shape of Y : {x_train_raw.shape}")
print(f"Test Shape of X : {x_test_raw.shape} and Test Shape of Y : {y_test_raw.shape}")
print(f"Validation Shape of X : {x_valid_raw.shape} and Validation Shape of Y : {y_valid_raw.shape}")
print(f"Shape of X : {x_data.shape} and Shape of Y : {y_data.shape}")

Train Shape of X : (304661, 1001) and Tranin Shape of Y : (304661, 1001)
Test Shape of X : (1200, 1001) and Test Shape of Y : (1200, 12)
Validation Shape of X : (3599, 1001) and Validation Shape of Y : (3599, 12)
Shape of X : (309460, 1001) and Shape of Y : (309460, 12)


### Calculate Sequence Positions to extracted from Original Sequence 

In [14]:
middle_index = (x_train_raw.shape[1] // 2) + 1

if CROP_STRATEGY == 'MID':
    STRAT_INEDX =middle_index - WINDOW_SIZE -1 
    END_INDEX =middle_index + WINDOW_SIZE 

if CROP_STRATEGY == 'END':
    STRAT_INEDX =middle_index - (WINDOW_SIZE*2) -1 
    END_INDEX =middle_index

# Testing data sample
# x_data =x_data.iloc[:1000]
# y_data =y_data.iloc[:1000]

print(f"Shape of X : {x_data.shape} and Shape of Y : {y_data.shape}")

Shape of X : (309460, 1001) and Shape of Y : (309460, 12)


### Apply Graph - Complex Network

In [16]:
def feature_extraction_graph(thresholdCN):
    metrics = []
    metrics.append(mean(thresholdCN.betweenness(directed=False, weights=None)))
    metrics.append(mean(thresholdCN.degree()))
    metrics.append(thresholdCN.assortativity_degree(directed=False))  # Returns the assortativity
    metrics.append(max(thresholdCN.degree()))
    metrics.append(min(thresholdCN.degree()))
    metrics.append(np.std(thresholdCN.degree()))  # Returns the strength (weighted degree)
    metrics.append(thresholdCN.average_path_length(directed=False, unconn=False))  # Average path length
    metrics.append(thresholdCN.transitivity_avglocal_undirected())  # local transitivity (clustering coefficient)
    metrics.append(thresholdCN.transitivity_undirected())  # global transitivity (clustering coefficient)
    metrics.append(cn.ecount())  # Counts the number of edges
    metrics.append(thresholdCN.motifs_randesu_no(size=3))
    metrics.append(thresholdCN.motifs_randesu_no(size=4))
    metrics.append(mean(thresholdCN.authority_score()))
    metrics.append(mean(thresholdCN.closeness(vertices=None, mode=ALL, cutoff=None, weights=None, normalized=True)))  # Calculates the closeness centralities of given vertices in a graph
    metrics.append(mean(thresholdCN.constraint(vertices=None, weights=None)))  # Calculates Burt's constraint scores for given vertices in a graph.
    metrics.append(mean(thresholdCN.count_multiple(edges=None)))  # Counts the multiplicities of the given edges.
    metrics.append(thresholdCN.density(loops=False))  # Calculates the density of the graph.
    metrics.append(thresholdCN.diameter(directed=False, unconn=False, weights=None))  # Calculates the diameter of the graph.
    metrics.append(mean(thresholdCN.eccentricity(vertices=None, mode=ALL)))  # Calculates the eccentricities of given vertices in a graph.
    metrics.append(mean(thresholdCN.edge_betweenness(directed=False, cutoff=None, weights=None)))  # Calculates or estimates the edge betweennesses in a graph.
    metrics.append(mean(thresholdCN.hub_score()))  # Calculates Kleinberg's hub score for the vertices of the graph.
    metrics.append(thresholdCN.maxdegree())  # Returns the maximum degree of a vertex set in the graph.
    metrics.append(mean(thresholdCN.neighborhood_size()))  # For each vertex specified by vertices, returns the number of vertices reachable from that vertex in at most order steps
    metrics.append(thresholdCN.radius())  # Calculates the radius of the graph.
    metrics.append(mean(thresholdCN.strength()))  # Returns the strength (weighted degree) of some vertices from the graph.
    metrics.append(cn.vcount())  # Counts the number of vertices.

    return metrics

def patterns(seq, win):
    """
    Generate k-mers: subsequences of length k
    contained in a biological sequence.
    """
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i + win > seqlen else i + win
        yield seq[i:j]
        if j == seqlen: break
    return

def complex_network(seq):
    """Generates complex network"""
    global name_seq, cn
    ksize = K_MERS_SIZE
    threshold = 3

    metrics = []
    cn = Graph()
    seq = seq.upper()
    
    for k in range(1, ksize + 1):
        cn = Graph()
        kmer = []
        for subseq in patterns(seq, k):  # Generates k pattern
            kmer.append(str(subseq))
        #print(kmer)
        vertices = np.unique(kmer)
        for vert in vertices:
            cn.add_vertices(vert)
        for i in range(len(kmer)-1):  # Position -1 -- Build the Network
            cn.add_edges([(kmer[i], kmer[i+1])])
        # print(summary(cn))
        metrics +=feature_extraction_graph(cn)
        #print('Result Matric' , metrics)
        #print('Size of Matric with k ' , k , '  Size ' , len(metrics))
    metrics_preprocessing = np.nan_to_num(metrics)     
    return metrics_preprocessing

def encode_using_graph(x_train_raw):
    truncated_df = x_train_raw.iloc[:,STRAT_INEDX :END_INDEX]
    truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    result = truncated_df['Sequence'].apply(complex_network)
    return result 



### Apply ANF Encoding 

In [193]:
# Apply ANF Encoding on Input Features
def apply_accumulated_nucle_frequency(seq):
    mapping = []
    A = 0
    C = 0
    T = 0
    G = 0
    for i, v in seq.items():
        if v == 'A':
            A += 1
            mapping.append(A / (i + 1))
        elif v == 'C':
            C += 1
            mapping.append(C / (i + 1))
        elif v == 'T' or v == 'U':
            T += 1
            mapping.append(T / (i + 1))
        else:
            G += 1
            mapping.append(G / (i + 1))
    padding = (MAX_LENGTH - len(mapping))
    mapping = np.pad(mapping, (0, padding), 'constant')
    return mapping

# # Encode X Features Based on Middle Index
def encode_using_anf(x_train_raw):
    truncated_df = x_train_raw.iloc[:,STRAT_INEDX :END_INDEX]
    print(truncated_df.shape)
    result = truncated_df.apply(apply_accumulated_nucle_frequency, axis=1)
    return result 

### Apply One Hot Encoding

In [194]:
# Create One Hot Encoding for RNA Sequence 
def one_hot_encode_sequence(seq):
    bases = 'ACGT'
    base_dict = {base: i for i, base in enumerate(bases)}
    one_hot = np.zeros((len(seq), len(bases)))
    for i, base in enumerate(seq):
        if base == 'N':
            continue
        one_hot[i, base_dict[base]] = 1
    return one_hot.flatten()


# Encode X Features Based on Middle Index
def encode_x_with_in_hot(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    concatenated_column= truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = truncated_df.assign(Sequence=concatenated_column)
    result = df_result['Sequence'].apply(one_hot_encode_sequence)
    return result 



### Apply Word2Vec Model

In [195]:
# Model Documentation - 
# Encode RNA sequences using Word2Vec embeddings
def encode_sequence(sequence , word2vec_model):
    embeddings = []
    for kmer in sequence:
        embeddings.append(word2vec_model.wv[kmer])
    return np.mean(embeddings, axis=0)

def applyKmersAndEncoding(seq):
    k = 3
    tokenized_sequences = [''.join(seq[i:i+k]) for i in range(0, len(seq)-k+1)]
    return tokenized_sequences
    


def encode_x_with_word2Vec(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    tokenized_sequences =  truncated_df['Sequence'].apply(applyKmersAndEncoding).tolist()
    
    # The result, tokenized_sequences, is a list of lists, where each inner list
    #  contains the k-mers of the corresponding RNA sequence from the truncated_df list. 
    embedding_size = 100
    word2vec_model = gensim.models.Word2Vec(tokenized_sequences, vector_size=embedding_size, window=5, min_count=1, workers=4)

    result = []
    for seq in tokenized_sequences:
        embedding = encode_sequence(seq , word2vec_model)
        result.append(embedding)
    return result 


### Apply One Hot with K mers

In [196]:
def applyOneHotEncoding(tokenized_sequences):
    encoded_sequences = []
    for seq in tokenized_sequences:
        encoded_sequences.append(one_hot_encode_sequence(seq))    
    # print("Encoded Sequence ")
    # print(encoded_sequences)
    return np.array(encoded_sequences).flatten()

def applyKmersAndEncoding(seq):
    k=3
    tokens = [seq[i:i+k] for i in range(0, len(seq)-k+1)]
    return tokens
    

def encode_x_with_k_mer_one_hot_encoding(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    tokenized_sequences =  truncated_df['Sequence'].apply(applyKmersAndEncoding).tolist()
    
    # The result, tokenized_sequences, is a list of lists, where each inner list
    #  contains the k-mers of the corresponding RNA sequence from the truncated_df list. 
   

    result = []
    for seq in tokenized_sequences:
        embedding = applyOneHotEncoding(seq)
        result.append(embedding)
    return np.array(result) 

### Apply Multi RM K-mers Embedding 

In [198]:
import pickle
model = pickle.load(open("C:/Users/shashi.vish/Documents/Shashi/Education/HigherEducation/NUS/Capstone Project/Git\MultiRM/Embeddings/embeddings_12RM.pkl", 'rb'))

def applyMultiRmEncoding(tokenized_sequences):
    embeddings = []
    for kmer in tokenized_sequences:
        embeddings.append(model[kmer])
    return np.mean(embeddings, axis=0)

def applyKmersAndEncoding(seq):
    k=3
    tokens = [seq[i:i+k] for i in range(0, len(seq)-k+1)]
    return tokens
    

def encode_x_with_k_mer_multi_rm(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    tokenized_sequences =  truncated_df['Sequence'].apply(applyKmersAndEncoding).tolist()

    result = []
    for seq in tokenized_sequences:
        embedding = applyMultiRmEncoding(seq)
        result.append(embedding)
    return result 


### Perform Encoding of X Features

In [21]:
# Apply Encoding Method
if ENCODING_METHOD == 1:
    x_encoded = encode_using_anf(x_data).tolist()
if ENCODING_METHOD == 3:
    x_encoded = encode_using_graph(x_data).tolist()
    #x_encoded = encode_using_graph(x_data).tolist()
if ENCODING_METHOD == 2:
    x_train = pd.DataFrame()
    x_train['one_hot_encoded'] = encode_x_with_in_hot(x_data)
    x_encoded = np.array(x_train['one_hot_encoded'].tolist())
    INPUT_DIMENSION = 4
if ENCODING_METHOD == 4:
     x_encoded = encode_x_with_word2Vec(x_data)
if ENCODING_METHOD == 5:
     x_encoded = encode_x_with_k_mer_one_hot_encoding(x_data)     
if ENCODING_METHOD == 6:
     x_encoded = encode_x_with_k_mer_multi_rm(x_data)   



# print("Shape of Encoded Data : " ,x_encoded.shape)
# print("Sample Records : ", x_encoded[0])  
  

C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_41816\1323896942.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)


In [22]:
# Persist Feature to a File.
# import csv
# with open('Complext_Network_Feature_Validation.csv', 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile , delimiter = ',')
#     for row in x_encoded:
#         writer.writerow(row)

### Encode Each Y category to 0 to 12 classes.

In [201]:
#Decode Y to its Original Form . Additional Class - NonMoD added for Non Modified RNA Sequence 
RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]

def decode_y(row):   
    decoded = ''
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            decoded = RMEncoding[index]
    
    if decoded == '':
        return 0
    
    return decoded 

y_encoded = y_data.apply(decode_y,axis=1)

y_encoded.value_counts()

0     154607
7      65178
11     52618
4      16341
6       3696
5       3207
10      3137
8       2447
3       2253
1       1878
12      1591
2       1471
9       1036
dtype: int64

In [202]:
#Code Reference : https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py

# PERFORM_DATA_BALANCING - if True then code will perform over sampling for each category.


### Split Data into Train and Test

In [203]:

# Train set
X_train, X_temp, y_train, y_temp = train_test_split(x_encoded, y_encoded, test_size=0.3, random_state=42)

# Test and Validation set
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


y_train.value_counts()

# Transform into Tensor
if FRAMEWORK == 'PYTORCH':
    X_train = torch.tensor(X_train , dtype=torch.float)
    y_train = torch.tensor(np.array(y_train), dtype=torch.long)  # Cross Entropy Expects Long Data Type

    X_test = torch.tensor(X_test , dtype=torch.float)
    y_test = torch.tensor(np.array(y_test) , dtype=torch.long)

    X_valid = torch.tensor(X_valid , dtype=torch.float)
    y_valid = torch.tensor(np.array(y_valid)  , dtype=torch.long)

    print("Shape of X : " , X_train[0].shape)



Shape of X :  torch.Size([19, 12])


### Re-Sample Data Using RandomOverSampler

In [204]:
if PERFORM_DATA_BALANCING:
    sm = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
    print(y_resampled.value_counts())
    

### Train using XGBoost 

In [205]:

if MODEL ==1:
    # Define XGBoost parameters
    params = {
        'objective': 'multi:softmax',
        'num_class': 13,
        'max_depth': 4,
        'n_jobs': -1,
        'enable_categorical': True
    }


    if PERFORM_DATA_BALANCING:
        dtrain = xgb.DMatrix(X_resampled, label=y_resampled)
        num_rounds = 100    
        model = xgb.train(params, dtrain, num_rounds)     
    else:
        dtrain = xgb.DMatrix(X_train, label=y_train)
        num_rounds = 100    
        model = xgb.train(params, dtrain, num_rounds) 

    dtest = xgb.DMatrix(X_test)
    y_pred = model.predict(dtest) 
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy: {accuracy:.2f}")


    dvalid = xgb.DMatrix(X_valid)
    # Make predictions on test set
    y_pred_val = model.predict(dvalid) 
    accuracy = accuracy_score(y_valid, y_pred_val)
    print(f"Validation Accuracy: {accuracy:.2f}")


    # Plot confusion matrix 
    cm = confusion_matrix(y_test, y_pred, labels=np.array(RMEncoding))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=np.array(RMEncoding))
    fig, ax = plt.subplots(figsize=(10,10))
    disp.plot(ax=ax)

    print("Accuracy of XGBoost  model: {:.2f}".format(accuracy))

### XgBoost with Hyper Paramter Tuning - RandomizedSearchCV

In [206]:
if MODEL == 2:
    # Create the XGBoost classifier
    xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=13, random_state=42)

    # Define the hyperparameter search space
    param_dist = {
        'learning_rate': [0.1, 0.01, 0.001],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight':   [1, 3, 5],
        'subsample': [0.5, 0.75, 1],
        'colsample_bytree': [0.5, 0.75, 1],
        'n_estimators': [100, 250, 500]
    }

    # Create the RandomizedSearchCV object
    random_search = RandomizedSearchCV(xgb_clf, param_dist, cv=3, n_iter=20, scoring='accuracy', verbose=1, n_jobs=-1, random_state=42)

    # Perform the randomized search
    random_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = random_search.best_params_
    print("Best hyperparameters:\n", best_params)

    # Train the classifier with the best hyperparameters
    best_xgb_clf = xgb.XGBClassifier(**best_params, objective='multi:softmax', num_class=13, random_state=42)
    best_xgb_clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = best_xgb_clf.predict(X_test)

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification report:\n", classification_report(y_test, y_pred))

### XgBoost with Hyper Paramter Tuning - GridSearchCV

In [207]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

if MODEL == 3:
    # Create the XGBoost classifier
    xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=13, random_state=42 )

    # Define the hyperparameter search space
    param_grid = {
        'learning_rate': [0.1, 0.01, 0.001],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.75, 1],
        'colsample_bytree': [0.5, 0.75, 1],
        'n_estimators': [100, 250, 500]
    }

    # Create the GridSearchCV object
    grid_search = GridSearchCV(xgb_clf, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

    # Perform the grid search
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("Best hyperparameters:\n", best_params)

    # Train the classifier with the best hyperparameters
    best_xgb_clf = xgb.XGBClassifier(**best_params, objective='multi:softmax', num_class=13, random_state=42)
    best_xgb_clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = best_xgb_clf.predict(X_test)

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification report:\n", classification_report(y_test, y_pred))

### Random Forest Classifier 

In [208]:
if MODEL == 4:
    # Train the Random Forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Evaluate the model
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification report:\n", classification_report(y_test, y_pred))


### Optimization with Optuna With XgBoost

In [209]:
def objective(trial):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)

    param = {
        "verbosity": 0,
        "objective": "multi:softmax",
        "num_class": 13 ,
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

if MODEL == 5:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=6000)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# Number of finished trials:  76
# Best trial:
#   Value: 0.6052047652900752
#   Params: 
#     booster: gblinear
#     lambda: 3.0789981964094497e-06
#     alpha: 2.468290506036041e-06
#     subsample: 0.8524524714955344
#     colsample_bytree: 0.498783948236617

## Save Model 

In [211]:
# import pickle

# TARGET_MODEL_PATH += '/xgboost_model.bin'

# ##dump the model into a file
# with open(TARGET_MODEL_PATH, 'wb') as f_out:
#     pickle.dump(bst, f_out) # write final_model in .bin file
#     f_out.close()  # close the file 
#     print("Exported Model Successfully")  